# Second Stage 2

In this stage, mainly conducted Topic modeling using LDA (Latent Dirichlet Allocation) on a sample size of the original data. As we used t-SNE to reduce feature dimension, sample size is limited due to the processing time. Maybe other method should be considered if the data size is big or should break down data into chunks for easier processing.

*Major difference with second stage: focusing on one sector's topic instead of everything*

In [19]:
import warnings
import lda
import logging
import dill 
import os
import pandas as pd
import numpy as np
import bokeh.plotting as bp
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from collections import Counter

warnings.filterwarnings("ignore", category=DeprecationWarning)
logging.getLogger("lda").setLevel(logging.WARNING)
code_dir = os.getcwd()
data_dir = code_dir.replace('code', 'data')

In [2]:
sub_body = dill.load(open(data_dir+'/#patent_title_lemma.pkd', 'rb'))

tokens = sub_body['title_lemma'].tolist()
titles = []
for token in tokens:
    titles.append(' '.join(token))

sub_body['title_lemma'] = titles
sub_body = sub_body.drop_duplicates(subset='title_lemma')
sub_body = sub_body.dropna(subset=['sector'])
sub_body.shape

(395873, 8)

In [ ]:
set(sub_body['sector'])

In [3]:
# set the sub dataframe that you want to focus on
chosen = 'Information Technology'
sub_body1 = sub_body[sub_body['sector'] == chosen]
sub_body2 = sub_body1[sub_body1['exec_dt']>'2010-01-01']
sub_body2.shape

(86222, 8)

In [70]:
sub_body2.head()

,exec_dt,or_name,title,ee_name_matched,ticker,sector,title_stem,title_lemma
4132,2010-01-12,"FULKERSON, TERRENCE M.","[pump, throat]",NORDSON CORPORATION,NDSN,Information Technology,"[pump, throat]",pump throat
4138,2010-01-12,"FULKERSON, TERRENCE M.","[pump, injector]",NORDSON CORPORATION,NDSN,Information Technology,"[pump, injector]",pump injector
4174,2010-06-10,"GAON, MARTIN, MR.","[adhesive, sensor, hot, melt, liquid, adhesives]",NORDSON CORPORATION,NDSN,Information Technology,"[adhes, sensor, hot, melt, liquid, adhes]",adhesive sensor hot melt liquid adhesive
4176,2010-07-01,"BONDESON, BENJAMIN J.","[cover, adhesive, dispensing, gun]",NORDSON CORPORATION,NDSN,Information Technology,"[cover, adhes, dispens, gun]",cover adhesive dispensing gun
4184,2010-07-01,"BONDESON, BENJAMIN J.","[thermally, insulated, applicator]",NORDSON CORPORATION,NDSN,Information Technology,"[thermal, insul, applic]",thermally insulated applicator


In [4]:
title_list = sub_body2['title_lemma'].tolist()
title_list[:3]

['pump throat', 'pump injector', 'adhesive sensor hot melt liquid adhesive']

In [97]:
# based on different data size, adjust parameters to make result more reasonable

cvectorizer = CountVectorizer(min_df=1, max_df=8000, max_features=20000, ngram_range=(3,3))
cvz = cvectorizer.fit_transform(sub_body2['title_lemma'])

n_topics = 20
n_iter = 1000
lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_model.fit_transform(cvz)

In [98]:
n_top_words = 10
topic_summaries = []

topic_word = lda_model.topic_word_  # get the topic words
vocab = cvectorizer.get_feature_names()

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(', '.join(topic_words))

In [8]:
# with open("file.txt", "w") as output:
#     output.write(str(topic_summaries))

In [99]:
tsne_model = TSNE(n_components=2, random_state=0, learning_rate=30, 
                  n_iter=300, angle=0.8, init='pca')
tsne_lda = tsne_model.fit_transform(X_topics)

In [100]:
doc_topic = lda_model.doc_topic_
lda_keys = []
for i, tweet in enumerate(sub_body2['title']):
    lda_keys += [doc_topic[i].argmax()]

In [ ]:
# plot_lda = bp.figure(plot_width=700, plot_height=600, title="LDA topic visualization",
#     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
#     x_axis_type=None, y_axis_type=None, min_border=1)

In [101]:
Counter(lda_keys)

Counter({0: 47682,
         1: 2140,
         2: 2051,
         3: 2109,
         4: 2263,
         5: 2216,
         6: 2047,
         7: 2158,
         8: 2231,
         9: 2231,
         10: 2100,
         11: 1995,
         12: 1764,
         13: 1886,
         14: 1964,
         15: 1768,
         16: 1986,
         17: 1844,
         18: 1819,
         19: 1968})

In [102]:
lda_df = pd.DataFrame(tsne_lda, columns=['x','y'])
lda_df['title'] = sub_body2['title_lemma'].tolist()
lda_df['ticker'] = sub_body2['ticker'].tolist()
lda_df['topic'] = lda_keys
lda_df['topic'] = lda_df['topic'].map(int)
lda_df.head()

,x,y,title,ticker,topic
0,0.329216,-0.930769,pump throat,NDSN,0
1,0.329216,-0.930769,pump injector,NDSN,0
2,0.329216,-0.930769,adhesive sensor hot melt liquid adhesive,NDSN,0
3,0.329216,-0.930769,cover adhesive dispensing gun,NDSN,0
4,0.329216,-0.930769,thermally insulated applicator,NDSN,0


In [12]:
colormap = np.array(["#6d8dca", "#69de53", "#723bca", "#c3e14c", "#c84dc9", "#68af4e", "#6e6cd5",
"#e3be38", "#4e2d7c", "#5fdfa8", "#d34690", "#3f6d31", "#d44427", "#7fcdd8", "#cb4053", "#5e9981",
"#803a62", "#9b9e39", "#c88cca", "#e1c37b", "#34223b", "#bdd8a3", "#6e3326", "#cfbdce", "#d07d3c",
"#52697d", "#7d6d33", "#d27c88", "#36422b", "#b68f79"])

In [ ]:
# plot_lda.scatter(source=lda_df, x='x', y='y', color=colormap[lda_keys])
# hover = plot_lda.select(dict(type=HoverTool))
# hover.tooltips={"title":"@title", "topic":"@topic", "ticker":"@ticker"}
# show(plot_lda)

# visualization of topics using pyLDAvis

In [103]:
lda_df['len_docs'] = lda_df['title'].apply(lambda x: len(x.split(' ')))

In [14]:
def prepareLDAData():
    data = {
        'vocab': vocab,
        'doc_topic_dists': lda_model.doc_topic_,
        'doc_lengths': list(lda_df['len_docs']),
        'term_frequency':cvectorizer.vocabulary_,
        'topic_term_dists': lda_model.components_
    } 
    return data

In [104]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [105]:
ldadata = prepareLDAData()

In [106]:
prepared_data = pyLDAvis.prepare(mds='tsne', **ldadata)

In [107]:
pyLDAvis.save_html(prepared_data,'./interactive plots/{}.html'.format(chosen))

In [108]:
# number of items in each topic
c = Counter(lda_df['topic'])
c.most_common()

[(0, 47682),
 (4, 2263),
 (9, 2231),
 (8, 2231),
 (5, 2216),
 (7, 2158),
 (1, 2140),
 (3, 2109),
 (10, 2100),
 (2, 2051),
 (6, 2047),
 (11, 1995),
 (16, 1986),
 (19, 1968),
 (14, 1964),
 (13, 1886),
 (17, 1844),
 (18, 1819),
 (15, 1768),
 (12, 1764)]

## for each topic, check the company rankings

In [109]:
topic_company_rank_df = pd.DataFrame()
for i in range(n_topics):
    topic_df = lda_df[lda_df['topic']==i]
    top_companies = Counter(topic_df['ticker']).most_common(10)
    topic_company_rank_df[i] = top_companies

In [110]:
topic_company_rank_df.columns = topic_summaries

In [111]:
topic_summaries

['mobile computing device, analog to digital, system method reducing, to digital converter, method apparatus reducing, system method identifying, method system determining, reducing power consumption, system method dynamically, integrated development environment',
 'computer readable medium, system computer readable, method system computer, light emitting diode, handheld communication device, organic light emitting, high aspect ratio, metal oxide semiconductor, light emitting device, long term evolution',
 'handheld electronic device, mobile electronic device, semiconductor memory device, electronic device associated, device associated method, radio access technology, integrated circuit design, mobile telecommunication system, non volatile semiconductor, distributed computing system',
 'dispersed storage network, encoded data slice, shallow trench isolation, data dispersed storage, time of flight, built in self, distributed storage network, automatic gain control, method use therewith,

In [112]:
topic_company_rank_df

,"mobile computing device, analog to digital, system method reducing, to digital converter, method apparatus reducing, system method identifying, method system determining, reducing power consumption, system method dynamically, integrated development environment","computer readable medium, system computer readable, method system computer, light emitting diode, handheld communication device, organic light emitting, high aspect ratio, metal oxide semiconductor, light emitting device, long term evolution","handheld electronic device, mobile electronic device, semiconductor memory device, electronic device associated, device associated method, radio access technology, integrated circuit design, mobile telecommunication system, non volatile semiconductor, distributed computing system","dispersed storage network, encoded data slice, shallow trench isolation, data dispersed storage, time of flight, built in self, distributed storage network, automatic gain control, method use therewith, sidewall image transfer","system method using, system method implementing, system method device, mobile data network, management system method, optical transport network, thermal interface material, system method generating, image forming device, system method secure","flash memory device, printed circuit board, direct memory access, natural language processing, intermediate transfer member, system method automatically, self aligned contact, system on chip, flash memory cell, service level agreement","dc dc converter, method system apparatus, local area network, wireless local area, method apparatus providing, plasma processing system, solid state drive, end to end, software defined network, digital right management","apparatus associated method, system method detecting, method apparatus controlling, phase change ink, device to device, method apparatus generating, radio communication system, system method facilitating, optical communication system, associated method facilitating","cloud computing environment, data storage system, system method generating, system method performing, method system managing, system method dynamic, apparatus method system, method system processing, method system detecting, method system controlling","system method providing, graphical user interface, portable electronic device, wireless communication system, wireless communication network, method apparatus providing, method system providing, method operation thereof, method program system, ink based digital","method apparatus system, system method managing, graphic processing unit, application programming interface, silicon on insulator, bipolar junction transistor, semiconductor on insulator, based access control, content addressable memory, device control method","processor method system, integrated circuit device, integrated circuit package, method apparatus performing, method system instruction, question answer system, integrated circuit chip, semiconductor integrated circuit, method apparatus use, testing integrated circuit","wireless communication device, mobile wireless communication, near field communication, programmable gate array, electronic device including, field programmable gate, touch sensitive display, communication system providing, field communication nfc, portable electronic device","field effect transistor, mobile communication device, system method controlling, fin field effect, source drain region, effect transistor device, method apparatus use, method system creating, distributed processing system, integrated circuit ic","data processing device, replacement metal gate, data processing system, method apparatus processing, magnetic recording medium, high metal gate, metal oxide semiconductor, near eye display, hand held electronic, three dimensional 3d","computer program product, method computer program, system method computer, system computer program, method system computer, method apparatus computer, system method program, apparatus computer program